In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia(user_agent="python-wikipediaapi", language="ja")

p_wiki = wiki_wiki.page("Claude")
context = p_wiki.text[:500]

print(context)

In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "Anthropicは元OpenAIのメンバーらが設立した企業で、倫理的なAI開発を重視している。\n"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "AnthropicはGoogle傘下のAI研究部門で、対話型AIに特化している。\n"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは2021年にMeta（旧Facebook）が開発し、2022年に全世界で同時公開された。\n"
            "リリース当初は「クラウディアAI」という名前で知られていた。\n"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "2025年現在、Claude4が最新版である。"
        ),
        "context": context,
    },
])
df

In [ ]:
from myllmet.io_aws import BedrockChatClient
from myllmet.metrics import Faithfulness
from myllmet.trackers import ListTracker

client = BedrockChatClient("anthropic.claude-3-5-sonnet-20240620-v1:0")
tracker = ListTracker()
metrics = Faithfulness.from_clients(client, client)

metrics.set_tracker(tracker)

In [ ]:
for r in df.itertuples(index=False):
    score = metrics.score(question=r.question, answer=r.answer, context=r.context)
    print(score)


In [ ]:
df_std = tracker.to_pandas("standard")
df_std

In [ ]:
df_int = tracker.to_pandas("intermediates")
df_int

In [ ]:
from IPython.display import JSON

JSON(
    df_int.to_json(force_ascii=False)
)